In [141]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import PassiveAggressiveClassifier

In [43]:
fake_news = pd.read_csv("data/news.csv")
fake_news = fake_news.loc[:, ~fake_news.columns.str.contains('^Unnamed')]

In [93]:
fake_news.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [94]:
fake_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 3 columns):
title    6335 non-null object
text     6335 non-null object
label    6335 non-null object
dtypes: object(3)
memory usage: 148.6+ KB


In [95]:
# shows that the classes are balanced
fake_news["label"].value_counts()

REAL    3171
FAKE    3164
Name: label, dtype: int64

In [96]:
X = fake_news[["title", "text"]]
y = fake_news["label"]

In [132]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [133]:
X_train.head()

,title,text
4541,Statistical Propaganda: How many Syrians has U...,"October 31, 2016 - Fort Russ - Aleksandr Khro..."
2490,'Russia has no intention of attacking anyone -...,"October 28, 2016 112 While the Western press c..."
4928,"Trump, Clinton trade blows on terror and guns ...",While investigators try to piece together what...
5627,Clinton clearing primary field for potential 2...,Hillary Clinton appears to have scared away mu...
757,What Brian Williams’s Chopper Whopper Says Abo...,Embarrassing and infuriating: the NBC anchor’s...


In [134]:
y_train.head()

4541    FAKE
2490    FAKE
4928    REAL
5627    REAL
757     REAL
Name: label, dtype: object

In [135]:
X_train.head()

,title,text
4541,Statistical Propaganda: How many Syrians has U...,"October 31, 2016 - Fort Russ - Aleksandr Khro..."
2490,'Russia has no intention of attacking anyone -...,"October 28, 2016 112 While the Western press c..."
4928,"Trump, Clinton trade blows on terror and guns ...",While investigators try to piece together what...
5627,Clinton clearing primary field for potential 2...,Hillary Clinton appears to have scared away mu...
757,What Brian Williams’s Chopper Whopper Says Abo...,Embarrassing and infuriating: the NBC anchor’s...


In [136]:
title_vectorizer = TfidfVectorizer(stop_words="english", max_df=0.7)
title_tfidf = title_vectorizer.fit_transform(X_train["title"])
X_train["title"] = list(title_tfidf.toarray())

/Users/willwen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [138]:
text_vectorizer = TfidfVectorizer(stop_words="english", max_df=0.7)
text_tfidf = text_vectorizer.fit_transform(X_train["text"])
X_train["text"] = list(text_tfidf.toarray())

/Users/willwen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [152]:
hey = X_train["title"]
lol = hey.tolist()
lol

[array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 

In [142]:
pac = PassiveAggressiveClassifier(max_iter=50)
pac.fit(X_train, y_train)

ValueError: setting an array element with a sequence.

In [120]:
text = ["The quick brown fox jumped over the lazy dog."]
vectorizer = TfidfVectorizer()
vectorizer.fit(text)
lol = vectorizer.transform(text)
lol.toarray()

array([[0.30151134, 0.30151134, 0.30151134, 0.30151134, 0.30151134,
        0.30151134, 0.30151134, 0.60302269]])

In [82]:
print(vectorizer.vocabulary_)

{'the': 7, 'quick': 6, 'brown': 0, 'fox': 2, 'jumped': 3, 'over': 5, 'lazy': 4, 'dog': 1}


In [86]:
# the lowest idf is sent to the most frequently appearing word
print(vectorizer.idf_)

[1.69314718 1.28768207 1.28768207 1.69314718 1.69314718 1.69314718
 1.69314718 1.        ]


In [84]:
vector = vectorizer.transform([text[0]])
print(vector.toarray())

[[0.36388646 0.27674503 0.27674503 0.36388646 0.36388646 0.36388646
  0.36388646 0.42983441]]
